In [1]:
# Auto reload modules
%load_ext autoreload
%autoreload 2
#if you haven't installed vsf and just want to run this from the current directory, uncomment the following lines
import sys
sys.path.append('..')

### Download the demo data from box (80MB): 
+ #### Demo data: https://uofi.box.com/s/31wozq63qgqvg8012r1jdfj5mdchmmfp

### Visualize VSF

We have already estimated some neural VSFs for you in the demo dataset.  To visualize them, you can use the `vsf_show` function.

In [2]:
import os

# demo data directory
DEMO_DIR = "../demo_data" # change to your path
OBJECT = "brown_boot_moving" # choose from "brown_boot_fixed" or "brown_boot_moving"
# OBJECT = "rubber_fig_tall_angle00" # choose from "brown_boot_fixed" or "brown_boot_moving"
DATA_DIR = os.path.join(DEMO_DIR, 'datasets', OBJECT)
TRIAL = ""

In [3]:
# visualize VSF
from vsf.constructors import vsf_from_file
import torch

#estimated VSFs have been saved to the "outputs" folder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vsf = vsf_from_file(os.path.join(DEMO_DIR, "saved_vsfs", OBJECT, "neural_vsf.pt")).to(device)

from klampt import vis
from vsf.visualize.klampt_visualization import vsf_show
vis.init('PyQt')  #needed inside Jupyter Notebook to show an OpenGL window
vsf_show(vsf)
vis.clear()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Walkthrough neural VSF training and visualization

### Create a world and simulator

In [4]:
import numpy as np
import json
import klampt
from klampt.math import se3
from vsf.sim import QuasistaticVSFSimulator
from vsf.sensor.joint_torque_sensor import JointTorqueSensor
from vsf.sensor.punyo_dense_force_sensor import PunyoDenseForceSensor
from vsf.sim.klampt_world_wrapper import klamptWorldWrapper
from vsf.utils.klampt_utils import load_trimesh_preserve_vertices

# NOTE: This part of the code shows how to manually setup a simulation world with a robot, a tactile sensor and a deformable object,
#       and how to estimate the VSF from tactile data.
#       A script to automate this process from config files is in scripts/neural_vsf_estimate.py

# create a world for simulation and visualization
world = klamptWorldWrapper()

# add robot/mesh to the world
world.add_robot('kinova', os.path.join('../knowledge/robot_model', 'kinova_gen3.urdf'))
robot = world.world.robot(0)
ee_link = robot.link(robot.numLinks()-1)
punyo2ee = np.array(json.load(open(os.path.join('../knowledge/robot_model', 'punyo2end_effector_transform.json')))['punyo2ee'])

#this function should be used instead of native Klampt loaders due to a known Assimp configuration issue
mesh = load_trimesh_preserve_vertices(os.path.join('../knowledge/robot_model', 'punyo_mesh_partial.ply'))
world.add_geometry('punyo', mesh, 'deformable', ee_link.getName(), punyo2ee)

# adds the VSF object to the world, for visualization purposes
# hack for add object to the visualization without interfering with the simulator
world2 = klampt.WorldModel()
object = world2.makeRigidObject("object")
if os.path.exists(os.path.join(DATA_DIR, "object", "mesh.obj")):
    object.geometry().loadFile(os.path.join(DATA_DIR, "object", "mesh.obj"))
elif os.path.exists(os.path.join(DATA_DIR, "object", "color_img.jpg")) and os.path.exists(os.path.join(DATA_DIR, "object", "depth_img.png")):
    # visualize the object with a heightmap (projected from RGB-D images)
    import imageio
    rgb_image = imageio.imread(os.path.join(DATA_DIR, "object", "color_img.jpg"))
    depth_image = imageio.imread(os.path.join(DATA_DIR, "object", "depth_img.png"))
    depth_scale = 1000.0
    depth_trunc = 2.0

    intrinsic = json.load(open(os.path.join(DATA_DIR, "object", "intrinsic.json")))
    extrinsic = json.load(open(os.path.join(DATA_DIR, "object", "extrinsic.json")))
    extrinsic = np.array(extrinsic['cam2world'])

    bmin, bmax = np.load(os.path.join(DATA_DIR, "object", "aabb.npy"))
    vp = klampt.Viewport()
    vp.setPose(*se3.from_homogeneous(extrinsic))
    vp.w, vp.h = rgb_image.shape[1], rgb_image.shape[0]
    vp.fx, vp.fy = intrinsic['fx'], intrinsic['fy']
    vp.cx, vp.cy = intrinsic['cx'], intrinsic['cy']

    # remove points beyond the truncation distance    
    depth_image[depth_image > depth_trunc*depth_scale] = 0

    # create a heightmap from the rgbd image
    hm_data = klampt.Heightmap()
    hm_data.setViewport(vp)
    hm_data.setHeightImage(depth_image, 1/depth_scale)
    hm_data.setColorImage(rgb_image)
    object.geometry().setHeightmap(hm_data)
    
else:
    aabb = np.load(os.path.join(DATA_DIR, "object", "aabb.npy"))
    object.geometry().setTriangleMesh(klampt.model.create.primitives.bbox(*aabb).getTriangleMesh())
    

# initialize sensors
sensors = [PunyoDenseForceSensor('punyo_force', 'punyo')] # add sensor named 'punyo_force' and attach it to the mesh named 'punyo'
# sensors = [JointTorqueSensor('kinova_joint_torques','kinova',[robot.link(i).name for i in range(1,8)])]

# create simulator
sim = QuasistaticVSFSimulator(world, sensors)

WorldModel::LoadRobot: ../knowledge/robot_model/kinova_gen3.urdf
URDFParser: Link size: 9
URDFParser: Joint size: 9
URDFParser: Done loading robot file ../knowledge/robot_model/kinova_gen3.urdf
ManagedGeometry: loaded ../demo_data/datasets/brown_boot_moving/object/mesh.obj in time 0.648093s


### Load tactile dataset and visualize it

In [5]:
import dacite
from vsf.dataset.constructors import dataset_from_config, DatasetConfig

from vsf.utils.config_utils import load_config_recursive
# dataset_config = load_config_recursive(os.path.join('../demo_data/datasets/joint_torques_dataset_config.yaml'))
dataset_config = load_config_recursive(os.path.join('../demo_data/datasets/punyo_dataset_config.yaml'))
dataset_config = dacite.from_dict(DatasetConfig, dataset_config)
dataset_config.path = os.path.join('../demo_data/datasets', OBJECT, TRIAL)
dataset = dataset_from_config(dataset_config)

In [6]:
from klampt import vis
vis.init('PyQt')  #needed inside Jupyter Notebook to show an OpenGL window
vis.clear()
vis.add("world", world.world)

# add table
# the table are for visualization only, not added to the simulator to avoid extra computation in the simulation
from klampt.model.create import box
b1 = box(1.,1.5,1.1,center=(0.25,0,-0.55),type='GeometricPrimitive')
vis.add("table", b1, hide_label=True)
b2 = box(0.7,0.45,0.12,center=(0.3,0.4,0),type='GeometricPrimitive')
vis.add("box", b2, hide_label=True)

# add visualization object to show its estimated pose
vis.add("object", object)
vis.show()

import time
for i in range(len(dataset)):
    seq = dataset[i]
    for frame in seq:
        control = {}
        control['kinova'] = frame['kinova_state']
        control['punyo'] = frame['punyo_state']

        # step simulation
        vis.lock()
        sim.step(control, 0.1)
        object.setTransform(*se3.from_ndarray(frame['object_state'])) # object mesh is for visualization only, so not added to the simulator
        vis.unlock()

        time.sleep(2/len(seq)) # visualize each sequence for 2 second

    sim.reset()
    if i >= 5:
        break
vis.clear()

***  klampt.vis: using Qt6 as the visualization backend  ***
Loading sequence ../demo_data/datasets/brown_boot_moving/seq_000


vis: creating GL window
######### QtGLWindow setProgram ###############
#########################################
klampt.vis: Making window 0
#########################################
######### QtGLWindow Initialize GL ###############
QtGLWindow.resizeGL: called when invisible?
QtGLWindow.paintGL: called while invisible?
TriMeshTopology: mesh has 9 triangles with duplicate neighbors!
  Triangle range 273 to 924
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 104 triangles with duplicate neighbors!
  Triangle range 3828 to 4274
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 271 triangles with duplicate neighbors!
  Triangle range 80 to 3362
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 635 triangles with duplicate neighbors!
  Triangle range 29 to 3626
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 587 triangles with duplicate neighb

#########################################
klampt.vis: Window 0 close
#########################################
#########################################
klampt.vis: Making window 0
#########################################
#########################################
klampt.vis: Window 0 close
#########################################
#########################################
klampt.vis: Making window 0
#########################################
#########################################
klampt.vis: Window 0 close
#########################################
#########################################
klampt.vis: Making window 0
#########################################
#########################################
klampt.vis: Window 0 close
#########################################
#########################################
klampt.vis: Making window 0
#########################################
#########################################
klampt.vis: Window 0 close
#######################################

In [7]:
# create VSF model
import torch
from vsf.constructors import vsf_from_box, vsf_from_mesh, vsf_from_rgbd

# create vsf from a bounding box
# aabb = np.load(os.path.join(DATA_DIR, "object", "aabb.npy"))
# vsf = vsf_from_box(aabb[0], aabb[1], type='neural')

# create vsf from a mesh -- this will do a better job estimating stiffness at the object boundaries
vsf = vsf_from_mesh(os.path.join(DATA_DIR, "object", "mesh.obj"))

# create vsf from a heightmap
# import imageio
# rgb_image = imageio.imread(os.path.join(DATA_DIR, "object", "color_img.jpg"))
# depth_image = imageio.imread(os.path.join(DATA_DIR, "object", "depth_img.png"))
# depth_scale = 1000.0
# depth_trunc = 2.0

# intrinsic = json.load(open(os.path.join(DATA_DIR, "object", "intrinsic.json")))
# intrinsic = np.array([[intrinsic['fx'], 0, intrinsic['cx']],
#                       [0, intrinsic['fy'], intrinsic['cy']],
#                       [0, 0, 1]])
# extrinsic = json.load(open(os.path.join(DATA_DIR, "object", "extrinsic.json")))
# extrinsic = np.array(extrinsic['cam2world'])
# bmin, bmax = np.load(os.path.join(DATA_DIR, "object", "aabb.npy"))

# vsf = vsf_from_rgbd(rgb_image, depth_image, bmin, bmax, intrinsic, extrinsic, depth_scale=depth_scale, depth_trunc=depth_trunc, type='neural')


# move vsf to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vsf = vsf.to(device)

# add vsf to the scene
sim.add_deformable('object',vsf)

Computing SDF
Computing SDF Done


## Run estimation

The following code runs a batch estimation over the whole dataset.  Note that training is performed by randomizing over sequences in the dataset.

In [8]:
# import dacite
# from vsf.sensor.constructors import SensorConfig, CalibrationConfig, calibration_from_config, BaseCalibrator

# calibrators_configs = {
#     'kinova_joint_torques': {
#         'type': 'tare',
#         # 'num_samples': 20,
#         'num_samples': 20,
#     }
# }

# calibrators = { k:calibration_from_config(dacite.from_dict(CalibrationConfig,v,config=dacite.Config(strict=True))) 
#                 for (k,v) in calibrators_configs.items() }

In [15]:
# Batch VSF estimation demo
from vsf.estimator.neural_vsf_estimator import NeuralVSFEstimator, NeuralVSFEstimatorConfig

# create estimator
estimator_config = NeuralVSFEstimatorConfig(lr=1e-3,
                                            regularizer_samples=500,
                                            regularizer_scale=1e-9,
                                            max_epochs=500,
                                            down_sample_rate=1)
                                            
estimator = NeuralVSFEstimator(estimator_config)

print("Starting batch estimation, using device",vsf.device)
estimator.batch_estimate(sim, vsf, dataset, dataset_config)

Starting batch estimation, using device cuda:0


The following code runs an online estimation for each sequence in the dataset.  This does have a risk of catastrophic forgetting, since there is no memory replay buffer (as in Point VSF online estimators).

In [14]:
# Online VSF estimation demo

# create estimator
estimator_config = NeuralVSFEstimatorConfig(lr=1e-3,
                                            regularizer_samples=500,
                                            regularizer_scale=1e-9)
estimator = NeuralVSFEstimator(estimator_config)

estimator.online_init(sim, vsf)

sensor_keys = dataset_config.sensor_keys
control_keys = dataset_config.control_keys

dt = 0.1
for i in range(len(dataset)):
    print("Beginning sequence",i)
    seq = dataset[i]
    sim.reset()
    estimator.online_reset(sim)
    for frame in seq:
        control, observation = {}, {}
        for k in sensor_keys:
            observation[k] = frame[sensor_keys[k]]
        for k in control_keys:
            control[k] = frame[control_keys[k]]

        sim.step(control, dt)
        loss = estimator.online_update(sim, dt, observation)
        print("Loss",loss)


Beginning sequence 0


AssertionError: 

In [16]:
# visualize VSF
from vsf.visualize.klampt_visualization import vsf_show
vsf_show(vsf)

vsf_show: Using the following stiffness values: [0.07224871776998043, 7420.5477900539445, 14841.023331390119, 22261.498872726293, 29681.974414062468]
klampt.vis: auto-fitting camera to scene.


Unrecognized OpenGL version
Unrecognized OpenGL version


In [17]:
# save to disk        
os.makedirs(os.path.join(DEMO_DIR, "saved_vsfs", OBJECT), exist_ok=True)
vsf.save(os.path.join(DEMO_DIR, "saved_vsfs", OBJECT, "neural_vsf_playground.pt"))


### Neural VSF to Point VSF


In [27]:
from vsf.constructors import vsf_from_vsf, PointVSFConfig

voxel_size = 3e-3
point_config = PointVSFConfig(voxel_size=voxel_size)
point_vsf = vsf_from_vsf(point_config, vsf)
vsf_show(point_vsf)

Setting voxel grid resolution 0.005341417590189599
vsf_show: Using the following stiffness values: [1.1592156852202606e-06, 0.00021194749569986016, 0.00042273575672879815, 0.0006335240323096514, 0.0008443123078905046]
klampt.vis: auto-fitting camera to scene.


Unrecognized OpenGL version
Unrecognized OpenGL version


### Point VSF to Neural VSF

In [41]:
from vsf.constructors import vsf_from_vsf, NeuralVSFConfig

neural_config = NeuralVSFConfig()

convert_config = NeuralVSFEstimatorConfig(lr=2e-4,
                                          regularizer_samples=1000,
                                          regularizer_scale=1e2,
                                          max_epochs=500)
neural_vsf = vsf_from_vsf(neural_config, point_vsf, convert_config)
vsf_show(neural_vsf)

Voxel size:  tensor(0.0030)


100%|██████████| 500/500 [05:08<00:00,  1.62it/s]


vsf_show: Using the following stiffness values: [0.043909826129674916, 5607.465451900842, 11214.886993975555, 16822.308536050266, 22429.73007812498]
klampt.vis: auto-fitting camera to scene.


Unrecognized OpenGL version
Unrecognized OpenGL version
